# Zero to Snowflake - Apps and Collaboration

**Asset:** Zero to Snowflake - Getting Started with Snowflake  
**Version:** v1  
**Copyright(c):** 2025 Snowflake Inc. All rights reserved.

## アプリケーションとコラボレーション
1. Snowflake Marketplaceからの気象データ取得
2. アカウントデータと気象ソースデータの統合
3. Safegraph POIデータの探索
4. SnowflakeでのStreamlitの紹介

まず、セッションクエリタグを設定

In [ ]:
ALTER SESSION SET query_tag = '{"origin":"sf_sit-is","name":"tb_101_v2","version":{"major":1, "minor":1},"attributes":{"is_quickstart":0, "source":"tastybytes", "vignette": "apps_and_collaboration"}}';

ワークシートコンテキストを設定

In [ ]:
USE DATABASE tb_101;
USE ROLE accountadmin;
USE WAREHOUSE tb_de_wh;

## 1. Snowflake Marketplaceからの気象データ取得

**ユーザーガイド:** https://docs.snowflake.com/en/user-guide/data-sharing-intro

ジュニアアナリストのBenは、天気が米国のフードトラック売上にどのような影響を与えるかについてより良い洞察を得たいと考えています。このために、Snowflake Marketplaceを使用してアカウントに気象データを追加します。

Snowflake Marketplaceは、第三者のデータ、アプリケーション、AI製品の幅広い種類を発見し、アクセスできる中央ハブを提供します。

### Weather Sourceデータを取得する手順:
1. アカウントレベルからaccountadminを使用していることを確認
2. ナビゲーションメニューから「データ製品」ページに移動
3. 検索バーに「Weather Source frostbyte」と入力
4. 「Weather Source LLC: frostbyte」リストを選択し、「取得」をクリック
5. 「オプション」をクリックしてオプションセクションを展開
6. データベース名を「ZTS_WEATHERSOURCE」に変更
7. 「PUBLIC」にアクセスを許可
8. 「完了」を押す

アナリストロールに切り替え

In [ ]:
USE ROLE tb_analyst;

## 2. アカウントデータと気象ソースデータの統合

Weather Sourceシェアからのデータで生の位置データを調和させる前に、データシェアを直接クエリして、作業しているデータの感覚を得ることができます。

In [ ]:
SELECT 
    DISTINCT city_name,
    AVG(max_wind_speed_100m_mph) AS avg_wind_speed_mph,
    AVG(avg_temperature_air_2m_f) AS avg_temp_f,
    AVG(tot_precipitation_in) AS avg_precipitation_in,
    MAX(tot_snowfall_in) AS max_snowfall_in
FROM zts_weathersource.onpoint_id.history_day
WHERE country = 'US'
GROUP BY city_name;

生の国データをWeather Sourceデータシェアの歴史的日次気象データと結合するビューを作成

In [ ]:
CREATE OR REPLACE VIEW harmonized.daily_weather_v
COMMENT = 'TastyBytesサポート都市にフィルタリングされたWeather Source日次履歴'
    AS
SELECT
    hd.*,
    TO_VARCHAR(hd.date_valid_std, 'YYYY-MM') AS yyyy_mm,
    pc.city_name AS city,
    c.country AS country_desc
FROM zts_weathersource.onpoint_id.history_day hd
JOIN zts_weathersource.onpoint_id.postal_codes pc
    ON pc.postal_code = hd.postal_code
    AND pc.country = hd.country
JOIN raw_pos.country c
    ON c.iso_country = hd.country
    AND c.city = hd.city_name;

2022年2月のハンブルクの平均日次気象温度をクエリ

結果を線グラフとして可視化する場合:
- チャートタイプ：線グラフ
- X軸：DATE_VALID_STD
- Y軸：AVERAGE_TEMP_F

In [ ]:
SELECT
    dw.country_desc,
    dw.city_name,
    dw.date_valid_std,
    AVG(dw.avg_temperature_air_2m_f) AS average_temp_f
FROM harmonized.daily_weather_v dw
WHERE dw.country_desc = 'Germany'
    AND dw.city_name = 'Hamburg'
    AND YEAR(date_valid_std) = 2022
    AND MONTH(date_valid_std) = 2
GROUP BY dw.country_desc, dw.city_name, dw.date_valid_std
ORDER BY dw.date_valid_std DESC;

日次気象ビューと注文ビューを組み合わせて、気象による日次売上ビューを作成

In [ ]:
CREATE OR REPLACE VIEW analytics.daily_sales_by_weather_v
COMMENT = '日次気象指標と注文データ'
AS
WITH daily_orders_aggregated AS (
    SELECT
        DATE(o.order_ts) AS order_date,
        o.primary_city,
        o.country,
        o.menu_item_name,
        SUM(o.price) AS total_sales
    FROM
        harmonized.orders_v o
    GROUP BY ALL
)
SELECT
    dw.date_valid_std AS date,
    dw.city_name,
    dw.country_desc,
    ZEROIFNULL(doa.total_sales) AS daily_sales,
    doa.menu_item_name,
    ROUND(dw.avg_temperature_air_2m_f, 2) AS avg_temp_fahrenheit,
    ROUND(dw.tot_precipitation_in, 2) AS avg_precipitation_inches,
    ROUND(dw.tot_snowdepth_in, 2) AS avg_snowdepth_inches,
    dw.max_wind_speed_100m_mph AS max_wind_speed_mph
FROM
    harmonized.daily_weather_v dw
LEFT JOIN
    daily_orders_aggregated doa
    ON dw.date_valid_std = doa.order_date
    AND dw.city_name = doa.primary_city
    AND dw.country_desc = doa.country
ORDER BY 
    date ASC;

シアトル市場で大雨が売上数値にどのような影響を与えるかを分析

チャートタイプ：棒グラフ | X軸：MENU_ITEM_NAME | Y軸：DAILY_SALES

In [ ]:
SELECT * EXCLUDE (city_name, country_desc, avg_snowdepth_inches, max_wind_speed_mph)
FROM analytics.daily_sales_by_weather_v
WHERE 
    country_desc = 'United States'
    AND city_name = 'Seattle'
    AND avg_precipitation_inches >= 1.0
ORDER BY date ASC;

## 3. Safegraph POIデータの探索

Benは、フードトラックの位置での気象条件についてより多くの洞察を得たいと考えています。幸い、SafegraphはSnowflake Marketplaceで無料のPOI（関心地点）データを提供しています。

### このデータリストを使用する手順:
1. アカウントレベルからaccountadminを使用していることを確認
2. ナビゲーションメニューから「データ製品」ページに移動
3. 検索バーに「safegraph frostbyte」と入力
4. 「Safegraph: frostbyte」リストを選択し、「取得」をクリック
5. 「オプション」をクリックしてオプションセクションを展開
6. データベース名：「ZTS_SAFEGRAPH」
7. 「PUBLIC」にアクセスを許可
8. 「完了」を押す

In [ ]:
CREATE OR REPLACE VIEW harmonized.tastybytes_poi_v
AS 
SELECT 
    l.location_id,
    sg.postal_code,
    sg.country,
    sg.city,
    sg.iso_country_code,
    sg.location_name,
    sg.top_category,
    sg.category_tags,
    sg.includes_parking_lot,
    sg.open_hours
FROM raw_pos.location l
JOIN zts_safegraph.public.frostbyte_tb_safegraph_s sg 
    ON l.location_id = sg.location_id
    AND l.iso_country_code = sg.iso_country_code;

POIデータを気象データと照会して、2022年の米国で平均して最も風の強い上位3つの位置を見つける

In [ ]:
SELECT TOP 3
    p.location_id,
    p.city,
    p.postal_code,
    AVG(hd.max_wind_speed_100m_mph) AS average_wind_speed
FROM harmonized.tastybytes_poi_v AS p
JOIN
    zts_weathersource.onpoint_id.history_day AS hd
    ON p.postal_code = hd.postal_code
WHERE
    p.country = 'United States'
    AND YEAR(hd.date_valid_std) = 2022
GROUP BY p.location_id, p.city, p.postal_code
ORDER BY average_wind_speed DESC;

異なる気象条件下での売上パフォーマンスを直接比較

このクエリは、各トラックブランドの売上データを2つのバケットに分割します：
- 「穏やか」な日：その日の最大風速が20mph以下
- 「風の強い」日：20mphを超えた日

これにより、どのブランドが「気象耐性」があり、どのブランドが強風で売上が大幅に低下するかを即座に特定できます。

In [ ]:
WITH TopWindiestLocations AS (
    SELECT TOP 3
        p.location_id
    FROM harmonized.tastybytes_poi_v AS p
    JOIN
        zts_weathersource.onpoint_id.history_day AS hd
        ON p.postal_code = hd.postal_code
    WHERE
        p.country = 'United States'
        AND YEAR(hd.date_valid_std) = 2022
    GROUP BY p.location_id, p.city, p.postal_code
    ORDER BY AVG(hd.max_wind_speed_100m_mph) DESC
)
SELECT
    o.truck_brand_name,
    ROUND(
        AVG(CASE WHEN hd.max_wind_speed_100m_mph <= 20 THEN o.order_total END),
    2) AS avg_sales_calm_days,
    ZEROIFNULL(ROUND(
        AVG(CASE WHEN hd.max_wind_speed_100m_mph > 20 THEN o.order_total END),
    2)) AS avg_sales_windy_days
FROM analytics.orders_v AS o
JOIN
    zts_weathersource.onpoint_id.history_day AS hd
    ON o.primary_city = hd.city_name
    AND DATE(o.order_ts) = hd.date_valid_std
WHERE o.location_id IN (SELECT location_id FROM TopWindiestLocations)
GROUP BY o.truck_brand_name
ORDER BY o.truck_brand_name;

---
## Reset Script
---

In [ ]:
USE ROLE accountadmin;

ビューの削除

In [ ]:
DROP VIEW IF EXISTS harmonized.daily_weather_v;
DROP VIEW IF EXISTS analytics.daily_sales_by_weather_v;
DROP VIEW IF EXISTS harmonized.tastybytes_poi_v;

クエリタグの解除

In [ ]:
ALTER SESSION UNSET query_tag;

ウェアハウスの一時停止

In [ ]:
ALTER WAREHOUSE tb_de_wh SUSPEND;